## Text Segmentation


Text segmentation is the process of dividing written text into meaningful units, such as words, sentences, or topics. When segmentation bounndaries are well defined, text segmentation is simple. But in unstructured data there are no distinct boundaries and segmentation becomes tedious task.
<br><br>
In this notebook we will be extracting different segments like abstract, methodology, conclusion in a research paper. 




## Reading a pdf file

We need preprocessed text before we start segmentation, so we will be extracting text from pdf file first.

In [2]:
import re
import string

In [20]:
file_path='researchpaper.pdf'


In [21]:
import PyPDF2
pdfFileObject = open(file_path, 'rb')
pdfReader = PyPDF2.PdfFileReader(pdfFileObject)
count = pdfReader.numPages
text = ''
for i in range(count):
    page = pdfReader.getPage(i)
    text += page.extract_text()


In [22]:
def text_cleaning(text):
    text = text.lower()
    return text

In [23]:
cleantext= text_cleaning(text)

In [24]:
headings =['Introduction','Abstract', 'Methodology', 'Conclusion','References',"Proposed System","Conclusion and Future Work"]


We need to find how many times a certain heading occurs in a given scientific paper. If it occurs muliple times then we need to find the actual heading start point. If **introduction** appears more than one time then its occurance can be mentioned in other segment of paper like abstract, methodology and so on.

In [25]:
heading_occurance = [(heading,cleantext.count(heading.lower()) )for heading in headings]
heading_occurance

[('Introduction', 1),
 ('Abstract', 1),
 ('Methodology', 0),
 ('Conclusion', 1),
 ('References', 1),
 ('Proposed System', 3),
 ('Conclusion and Future Work', 1)]

`find_occurance` function returns indices where a given heading is present

In [26]:
def find_occurance(heading):
    occurance_indices = [word.start() for word in re.finditer(heading, cleantext)]
    return occurance_indices


In order to determine if a given portion of text is start of segment, we need to analyze fews words after the occurance of heading.
If there is no new line or if there is presence of punctuation marks like `.?,;` after the heading term or some words , then it can't be the start of text. It should be the only word or phrase  in a  line, to get identified as a heading.

In [27]:
def prob_next_word(index,heading):
    return cleantext[index:index+len(heading)+10]

In [28]:
def available_topics(headings):
    available_headings =[]
    headings = [heading.lower()for heading in headings]
    start_indices = []
    punctuations= string.punctuation 
    
    for heading in headings:
        frequency = cleantext.count(heading.lower())
        occurance_indices = find_occurance(heading)
        if len(occurance_indices) >0:
            for index in occurance_indices:
                next_words_seq = prob_next_word(index,heading)
                immediate_next_word = next_words_seq[len(heading):]

                length= len(immediate_next_word.replace(" ",""))
                is_start = ("\n" in next_words_seq) and (punctuations not in next_words_seq)
                if is_start:
                    start_indices.append(index)
                    available_headings.append(heading)
    return available_headings,start_indices
available_headings,start_indices= available_topics(headings)

Start of one segment is end for another segment. Inorder to find the ending of a segment, we choose segment whose starting point is nearest to the current segment's  out of all segment's starting points.

In [29]:
def nearest_higher_index(heading_index,start_indices):
    sorted_indices =  sorted(start_indices)
    end_index = sorted_indices.index(heading_index)+1
    if end_index == len(sorted_indices):
        end_index_val = len(cleantext)
    else:
        end_index_val =  sorted_indices[end_index]
    return end_index_val
    

For extracting sections like introduction, abstract we need to find its starting point and ending point.

In [30]:
def start_and_end_index(headings,start_indices):
    heading_prob_points = []
    for heading in available_headings:
        
        start_index_val = start_indices[available_headings.index(heading)]
        end_index_val =nearest_higher_index(start_index_val,start_indices)
        print((heading,start_index_val,end_index_val))
        heading_prob_points.append((heading,start_index_val,end_index_val))
    return heading_prob_points

In [31]:
heading_prob_points= start_and_end_index(headings,start_indices)

('introduction', 1249, 5584)
('abstract', 330, 1249)
('references', 20127, 23531)
('proposed system', 5584, 19391)
('conclusion and future work', 19391, 20127)


In [37]:
common_headings = ["introduction","abstract","conclusion and future work"]

In [48]:
user_heading = input("Enter heading of your choice ")
user_heading = user_heading.lower()
print(user_heading)

Enter heading of your choice abstract
abstract


In [49]:
if user_heading not in available_headings:
    print("The heading specified by you not present in research paper")
else:
    for heading in available_headings:
        index = available_headings.index(heading)
        prob_points = heading_prob_points[index]
        text = cleantext[prob_points[1]+len(heading):prob_points[2]]
        if heading == user_heading:
            print(heading.upper())
            print(text,"\n")
    

ABSTRACT

in this work, we address the problem
of spelling correction in the arabic lan-
guage utilizing the new corpus provided
by qalb (qatar arabic language bank)
project which is an annotated corpus of
sentences with errors and their corrections.
the corpus contains edit, add before, split,
merge, add after, move and other error
types. we are concerned with the rst four
error types as they contribute more than
90% of the spelling errors in the corpus.
the proposed system has many models to
address each error type on its own and then
integrating all the models to provide an
efcient and robust system that achieves
an overall recall of 0.59, precision of 0.58
and f1 score of 0.58 including all the error
types on the development set. our system
participated in the qalb 2014 shared task
ﬂautomatic arabic error correctionﬂ and
achieved an f1 score of 0.6, earning the
sixth place out of nine participants.
1  

